In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from paths import file_ex_atm_T_T850_yrly, path_results, path_samoc
from constants import abs_zero
from timeseries import GMST_timeseries
from timeseries import IterateOutputCESM
from xr_integrate import xr_surf_mean, xr_zonal_mean
from xr_DataArrays import generate_xr_AREA

In [ ]:
ds = xr.open_dataset(file_ex_atm_T_T850_yrly, decode_times=False)
AREA = generate_xr_AREA('atm')

In [ ]:
xr_surf_mean(ds['T'][-1,:,:], AREA=AREA)-273.15

In [ ]:
ds.coords

In [ ]:
zm = xr_zonal_mean(ds['T'][-1,:,:], AREA=AREA, dx=1, lat_name='lat')

In [ ]:
plt.plot(zm.lat_bins, zm + abs_zero) 

In [ ]:
(ds['T'][-1,:,:]-273.15).plot()

In [ ]:
%%time
gmst_ctrl = GMST_timeseries('ctrl')
gmst_rcp  = GMST_timeseries('rcp')

In [ ]:
plt.plot(gmst_ctrl.GMST.values, label='ctrl')
plt.plot(gmst_rcp.GMST.values,  label='RCP85')
plt.xlabel('time [years]')
plt.ylabel('GMST [$^\circ$C]')
plt.legend()
plt.tight_layout()
plt.savefig(f'{path_results}/GMST/GMST_rcp_ctrl')

In [ ]:
gmst_rcp.T_zonal.mean(dim='year').plot()

In [ ]:
N_ctrl = len(gmst_ctrl.coords['year'])
N_rcp  = len(gmst_rcp.coords['year'])
print(N_ctrl, N_rcp)
N = min(N_ctrl, N_rcp)

for i in range(N):
    plt.plot(gmst_rcp.coords['lat_bins'], 
             gmst_rcp.T_zonal[i,:]-gmst_ctrl.T_zonal[i,:], 
             color=plt.cm.rainbow(i/N), alpha=.4, lw=.5,
             label='')
for i in range(N):
    if (i-5)%10==0:
        plt.plot(gmst_rcp.coords['lat_bins'], 
                 gmst_rcp.T_zonal[i-5:i+5,:].mean(dim='year')-gmst_ctrl.T_zonal[i-5:i+5,:].mean(dim='year'), 
                 color=plt.cm.rainbow(i/N), alpha=1, lw=2, ls='--',
                 label=f'decade {int((i-5)/10)}')
plt.axhline(0, c='k', lw=.5)
plt.legend(ncol=4, frameon=False)
plt.ylabel('zonally averaged temp. RCP8.5 - CTRL [K]')
plt.xlabel('Latitude')
plt.xticks(np.arange(-90,91,30))
plt.tight_layout()
plt.savefig(f'{path_results}/GMST/T_zonal_rcp-ctrl')

In [ ]:
# trends
from xr_regression import xr_linear_trends_2D
# create an example dataset
da = xr.DataArray(np.random.rand(10,30,40), dims=['time', 'lat', 'lon'])
da_trend = xr_linear_trends_2D(da, ('lat', 'lon'))

In [ ]:
da_trend.plot()

In [ ]:
from xr_regression import atm_field_regression
trends_ctrl = atm_field_regression('ctrl')
trends_ctrl.to_netcdf(path=f'{path_results}/GMST/trend_ctrl.nc' , mode='w')

trends_rcp  = atm_field_regression('rcp')
trends_rcp.to_netcdf(path=f'{path_results}/GMST/trend_rcp.nc' , mode='w')

In [ ]:
trends_ctrl.plot()

In [ ]:
trends_rcp.plot()

In [ ]:
trends_ctrl

In [ ]:
(trends_rcp/abs(trends_ctrl)).plot(vmin=-8,vmax=10, cmap='RdBu_r')

In [ ]:
%%time
da_trend = xr_linear_trends_2D(da, ('lat', 'lon'))

In [ ]:
trend.plot()

In [ ]:
# difference between 
ds_last_ctrl = xr.open_dataset(f'{path_samoc}/ctrl/atm_yrly_T_T850_0266.nc', decode_times=False)
ds_last_rcp = xr.open_dataset(f'{path_samoc}/rcp/atm_yrly_T_T850_2066.nc', decode_times=False)
TEMP_diff = ds_last_rcp['T'][-1,:,:]-ds_last_ctrl['T'][-1,:,:]
im = TEMP_diff.plot()
plt.colorbar(im=im)

In [ ]:
z = np.array([0,1])
y = np.array([0,1,2])
x = np.array([0,1,2,3])
A = xr.DataArray(data=np.ones((2,3,4)),
                      coords={'z': z, 'y': y, 'x': x},
                      dims=('z', 'y', 'x') )
B = xr.DataArray(data=np.ones((3,4)),
                      coords={'y': y, 'x': x},
                      dims=('y', 'x') )

In [ ]:
C = xr.Dataset()

In [ ]:
C['B'] = B
C['A'] = A


In [ ]:
C